## Import

In [1]:
import os, random
import numpy as np
import pandas as pd

from ase.io import read #  Python 기반의 화학 소프트웨어 라이브러리

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from tqdm.auto import tqdm

np.set_printoptions(threshold=np.inf) # 배열을 출력할 때 출력 형식을 지정. 배열의 요소 수에 상관없이 모든 요소를 출력하도록 지정.

c:\Users\PC\miniconda3\envs\samsung\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(42) # Seed 고정

## Pre-Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = read('./data/train.xyz', format='extxyz', index=':') # 전체 데이터 불러오기
test = read('./data/test.xyz', format='extxyz', index=':')
sample = pd.read_csv('./data/sample_submission.csv')

In [4]:
print(f"The number of data: {len(train)}")
train[0]

The number of data: 22510


Atoms(symbols='N24Si24', pbc=True, cell=[8.52238831, 8.52238831, 8.52238831], forces=..., calculator=SinglePointCalculator(...))

In [ ]:
(train[6].get_positions()).shape

In [ ]:
train[0].get_forces().shape != (48, 3)

In [ ]:
train[1].get_forces().shape

In [ ]:
train[0].get_forces().shape

In [ ]:
type(train[0].get_total_energy())

In [ ]:
mole = train[0]

In [ ]:
mole[47]

Atom('Si', [2.55477637, 4.14399683, 0.91811246], index=47)

In [ ]:
sequence_train, symbols, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], [], []

for i in range(len(train)):
    mole = train[i] # 각 분자 - 22510개

    atoms = len(mole) # 원자 개수 - 48개
    sequence_train.append(atoms) # 22510개의 mole에 들어있는 atoms의 개수를 저장

    position = mole.get_positions() # 원자 위치 정보 -> (48, 3)
    force = mole.get_forces() # label 1 -> (48, 3)

    energy = mole.get_total_energy() # label 2 -> float 값 하나
    energies.append(energy)

    for j in range(len(mole)): # 각 원자에 대해 반복 -> 48회
        atom = mole[j] # i번 분자의 j번째 원자

        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        forces.append(force[j])

train_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':forces}) # sequence_train, symbols, energies 아직 사용 안 함
train_df.head()

,position_x,position_y,position_z,force
0,1.591737,4.200483,7.832245,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,"[-4.65938123, -0.77685475, -3.07403915]"


In [ ]:
sum(train_df['force'].apply(lambda x: len(x) != 3).astype(int))

0

In [ ]:
len(train[0])

In [ ]:
len(train_df)

1284975

In [ ]:
len(train_df) == len(train) * len(train[0])

In [8]:
sequence_test, positions_x, positions_y, positions_z = [], [], [], [] # 테스트에서도 동일하게 적용. 하지만 테스트에서는 xyz밖에 없음
forces = []
for i in range(len(test)):
    mole = test[i] # 각 분자

    atoms = len(mole) # 원자 개수
    sequence_test.append(atoms)

    position = mole.get_positions() # 원자 위치 정보
    force = mole.get_forces()

    for j in range(len(mole)): # 각 원자에 대해
        atom = mole[j]

        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        forces.append(force[j])
forces = None
test_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':forces})
test_df.head()

,position_x,position_y,position_z,force
0,9.671275,8.734431,6.151755,None
1,1.676806,2.238918,5.270450,None
2,10.358608,4.824889,9.174357,None
3,4.370620,5.391541,9.812298,None
4,2.453404,10.449967,9.906622,None


## [Force] Hyperparameter Setting

In [ ]:
# 하이퍼파라미터
input_size = 3  # feature 개수
hidden_size = 256
output_size = 3 # target 개수
num_epochs = 3
batch_size = 256
learning_rate = 0.001

## [Force] Dataset

In [ ]:
class ForceDataset(Dataset):
    def __init__(self, df, mode='test'):
        self.df = df
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        pos_x = self.df.loc[idx, 'position_x']
        pos_y = self.df.loc[idx, 'position_y']
        pos_z = self.df.loc[idx, 'position_z']

        inputs = [pos_x, pos_y, pos_z]

        if not self.mode == 'test':
            label = self.df.loc[idx, 'force']
            return inputs, label
        else:
            return inputs

In [ ]:
train_dataset = ForceDataset(train_df, 'train')
test_dataset = ForceDataset(test_df, 'test')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## [Force] Model

In [ ]:
class ForceModel(nn.Module):
    def __init__(self, input_size, hidden_size): # input_size = 3 -> xyz, output_size = 3 -> forces
        super(ForceModel, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size), # hidden_size = 256
            nn.BatchNorm1d(hidden_size), # 배치 정규화, 이는 eval 모드에서는 적용되지 않는다.
            nn.ReLU(), # 비선형성을 추가하면서도 계산이 빠르고 경사 소실 문제를 완화
            nn.Dropout(0.5),

            nn.Linear(hidden_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.01), # 입력이 음수일 때도 작은 기울기를 가지는 ReLU의 변형
            nn.Dropout(0.5),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64), #
            nn.ReLU(), # 비활성함수 이전에 배치정규화를 해주는 것이 일반적.
            nn.Dropout(0.5),

            nn.Linear(64, 3)
        )

    def forward(self, x):
        y = self.layers(x)

        return y

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"current device is {device}")

model = ForceModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss() # MSELoss 사용
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam

current device is cuda


## [Force] Train

In [ ]:
print("Training Start!")

model.train()
for epoch in range(num_epochs):
    print(f"{epoch+1}/{num_epochs} epoch..")
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

print("Training Complete!")

Training Start!
1/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

2/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

3/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

Training Complete!


## [Force] Inference

In [ ]:
print("Inference Start!")

model.eval()

preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)

        pred = outputs.detach().cpu().numpy()
        preds.extend(pred)

print("Inference Complete!")
len(preds)

Inference Start!


  0%|          | 0/1154 [00:00<?, ?it/s]

Inference Complete!


295234

## [Force] Submission

In [ ]:
test_df['force'] = preds # 예측 결과 저장

In [ ]:
test_df

,position_x,position_y,position_z,force
0,9.671275,8.734431,6.151755,"[0.012714688, 0.0064194333, -0.0049114088]"
1,1.676806,2.238918,5.270450,"[0.009695852, 0.0072880536, -0.0007058511]"
2,10.358608,4.824889,9.174357,"[0.012420569, 0.0064737373, -0.00449712]"
3,4.370620,5.391541,9.812298,"[0.012712338, 0.005861702, -0.004311632]"
4,2.453404,10.449967,9.906622,"[0.0067618014, 0.0090170335, 0.0007615231]"
...,...,...,...,...
295229,10.906604,1.917709,5.112100,"[0.012146747, 0.0063163284, -0.0034246566]"
295230,0.964534,0.435691,9.589554,"[0.01304807, 0.0053317044, -0.0042844103]"
295231,7.450363,2.964188,7.225830,"[0.011606228, 0.0066240923, -0.0033500595]"
295232,0.025578,9.331741,6.579088,"[0.007906787, 0.009996454, -0.0011956814]"


In [ ]:
sum(test_df['force'].apply(lambda x: len(x) != 3).astype(int))

0

In [ ]:
# 한 분자가 몇 개의 원자로 이루어져 있는지에 따라 범위를 생성
bundles_train, bundles_test = [], []

flag = 0
for size in sequence_train: # 각 mole을 이루는 atom의 개수-train
    bundles_train.append((flag, flag+size))
    flag += size

flag = 0
for size in sequence_test: # 각 mole을 이루는 atom의 개수-test
    bundles_test.append((flag, flag+size))
    flag += size

In [ ]:
preds_force = []

for start, end in bundles_test: # 시작과 끝. 예를 들어 train[0]은 시작 0번부터 끝 47번일 것
    preds_force.append(np.vstack(preds[start:end])) # 2차원 array로 저장

sample['force'] = preds_force
sample

,ID,energy,force
0,TEST_0000,0,"[[0.012714688, 0.0064194333, -0.0049114088], [..."
1,TEST_0001,0,"[[0.012255937, 0.0065041343, -0.0042652227], [..."
2,TEST_0002,0,"[[0.012147827, 0.006524095, -0.004112943], [0...."
3,TEST_0003,0,"[[0.012587041, 0.0064430013, -0.0047316076], [..."
4,TEST_0004,0,"[[0.013105195, 0.006347332, -0.0054614674], [0..."
...,...,...,...
4096,TEST_4096,0,"[[0.009327458, 0.008304927, -0.00032931505], [..."
4097,TEST_4097,0,"[[0.012546437, 0.0060378434, -0.0033117067], [..."
4098,TEST_4098,0,"[[0.012195604, 0.006121706, -0.002880563], [0...."
4099,TEST_4099,0,"[[0.012258429, 0.0061073904, -0.0029600875], [..."


## [Energy] Preprocessing

In [ ]:
train_df

,position_x,position_y,position_z,force
0,1.591737,4.200483,7.832245,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,"[-4.65938123, -0.77685475, -3.07403915]"
...,...,...,...,...
1284970,9.495734,3.501358,5.231880,"[2.449508, 21.06236726, 48.41937431]"
1284971,0.715535,1.041657,9.207530,"[-0.21919686, 1.74001551, 0.66362062]"
1284972,7.630116,3.670234,7.966260,"[0.79347178, -2.8472516, -0.91132274]"
1284973,0.177211,8.660930,7.082750,"[-0.20181181, -0.0042076, -0.14967853]"


In [ ]:
# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = train_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
train_df = train_df.drop('force', axis=1).join(force_df)

# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = test_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
test_df = test_df.drop('force', axis=1).join(force_df)
test_df.head()

,position_x,position_y,position_z,force_0,force_1,force_2
0,9.671275,8.734431,6.151755,0.012715,0.006419,-0.004911
1,1.676806,2.238918,5.270450,0.009696,0.007288,-0.000706
2,10.358608,4.824889,9.174357,0.012421,0.006474,-0.004497
3,4.370620,5.391541,9.812298,0.012712,0.005862,-0.004312
4,2.453404,10.449967,9.906622,0.006762,0.009017,0.000762


In [ ]:
train_df

,position_x,position_y,position_z,force_0,force_1,force_2
0,1.591737,4.200483,7.832245,-1.936480,-2.755401,0.908990
1,5.640802,2.305094,4.606757,1.770470,-0.173502,-1.993986
2,6.672786,8.483263,2.981881,-2.054887,-0.293816,-0.891738
3,1.908548,0.147931,1.741693,-0.892072,-0.814316,-1.364269
4,4.375650,6.837884,1.948188,-4.659381,-0.776855,-3.074039
...,...,...,...,...,...,...
1284970,9.495734,3.501358,5.231880,2.449508,21.062367,48.419374
1284971,0.715535,1.041657,9.207530,-0.219197,1.740016,0.663621
1284972,7.630116,3.670234,7.966260,0.793472,-2.847252,-0.911323
1284973,0.177211,8.660930,7.082750,-0.201812,-0.004208,-0.149679


In [ ]:
# 데이터프레임에서 값 추출
sequences_train = [train_df.iloc[start:end].values for start, end in bundles_train]
sequences_test = [test_df.iloc[start:end].values for start, end in bundles_test]

In [ ]:
len(sequences_train)

22510

## [Energy] Hyperparameter Setting

In [ ]:
input_size = 6  # feature 개수
hidden_size = 256
output_size = 1 # target 개수
num_epochs = 1
batch_size = 64
learning_rate = 0.001

## [Energy] Dataset

In [ ]:
# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
max_len = max(seq.shape[0] for seq in sequences_train) # 510
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_train]
# sequences_train의 하나씩(0번 분자)에 대해 (510 - 길이(48), 6) 크기의 0 array를 만들고, 그걸 seq(48,6)에 붙여준다. 그럼 모든 seq들이 max_len에 맞춰질 것.

# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_train = np.stack(padded_sequences) # padded_sequence는 (510, 6) 크기의 array가  22510개 있을 것. 그걸 np.stack으로 쌓아준다. -> (22510, 510, 6)
X_tensor_train = torch.tensor(padded_array_train, dtype=torch.float32) # 이제 x,y,z,force1,2,3이 feature가 된다. 22510개의 분자.
y_tensor_train = torch.tensor(energies, dtype=torch.float32).view(-1, 1) # 위에서 train energy는 분자별로 단 하나의 값이었다. 그걸 1자로 쭉 펴준다. 22510개의 에너지
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
max_len = max(seq.shape[0] for seq in sequences_test) # test에서도 동일하게 적용.
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_test]

# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_test = np.stack(padded_sequences)
X_tensor_test = torch.tensor(padded_array_test, dtype=torch.float32)
test_dataset = TensorDataset(X_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## [Energy] Model

In [ ]:
# BiLSTM 모델 정의
class EnergyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout_rate=0.5):
        super(EnergyModel, self).__init__()

        # Bidirectional LSTM with Dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True,
                            dropout=dropout_rate,
                            bidirectional=True) # bidirectional true만 해주면 가능

        # Bidirectional LSTM이므로 hidden_size 조정
        self.linear = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size), # lstm을 통과한 hidden size는 bidirectional이기 때문에 2배임
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size), # 여긴 렐루 배치 순서다. 바꿔야 할듯?
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        energy = self.linear(lstm_out[:, -1, :]) # 역시 마지막 시퀀스만 취함
        return energy

In [ ]:
# 모델, 손실 함수, 옵티마이저 초기화
model = EnergyModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


## [Energy] Train

In [ ]:
print("Training Start!!")

# 학습
model.train()
for epoch in range(num_epochs):
    print(f"{epoch+1}/{num_epochs} epoch..")
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

print("Training Complete!")

Training Start!!
1/1 epoch..


  0%|          | 0/352 [00:00<?, ?it/s]

Training Complete!


## [Energy] Inference

In [ ]:
print("Inference Start!")

model.eval()

preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        pred = outputs.detach().cpu().numpy()

        preds.extend(pred)

print("Inference Complete!")
len(preds)

Inference Start!


  0%|          | 0/65 [00:00<?, ?it/s]

Inference Complete!


4101

## [Energy] Submission

In [ ]:
preds = [pred.item() for pred in preds]
sample['energy'] = preds
sample

,ID,energy,force
0,TEST_0000,-61.111519,"[[0.012714688, 0.0064194333, -0.0049114088], [..."
1,TEST_0001,-61.111549,"[[0.012255937, 0.0065041343, -0.0042652227], [..."
2,TEST_0002,-61.111523,"[[0.012147827, 0.006524095, -0.004112943], [0...."
3,TEST_0003,-61.111523,"[[0.012587041, 0.0064430013, -0.0047316076], [..."
4,TEST_0004,-61.111492,"[[0.013105195, 0.006347332, -0.0054614674], [0..."
...,...,...,...
4096,TEST_4096,-61.111599,"[[0.009327458, 0.008304927, -0.00032931505], [..."
4097,TEST_4097,-61.111591,"[[0.012546437, 0.0060378434, -0.0033117067], [..."
4098,TEST_4098,-61.111572,"[[0.012195604, 0.006121706, -0.002880563], [0...."
4099,TEST_4099,-61.111561,"[[0.012258429, 0.0061073904, -0.0029600875], [..."


In [ ]:
sample.to_csv('baseline_submission.csv', index=False)